## Structured Tool

A structured tool represents an action an agent can take. It wraps any function you provide to let an agent easily interface with it. A structured tool is defined by: 
* `name` - tells the agent what tool to pick, name is important for information retrevial
* `description` - a short instruction manual that explains when and why the agent should use the tools
* `args_schema` - communicates the interface of the tool agent. Typically draws from teh wrapped function's signature
* `_run` and `_arun` functions - defines the tools inner workings

Here are some helpful resources for working with tools in Langchain: 
* https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#subclassing-the-basetool-class
* https://blog.langchain.dev/structured-tools/


Below is a basic example of us calling an agent and getting a basic response from a llm. We will attempt to override that answer with our own.

In [95]:
from langchain.tools import StructuredTool, Tool
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_openai_tools_agent
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field

with open('api_key.txt','r') as file:
    api_key = file.read()

# initialize llm object
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", 
                  temperature=0.2, 
                  api_key=api_key)


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}")
])

# lets you parse the response into a string, handles the differences of different llms
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "What is the color of the sky on the planet Hexar?"})

"I'm sorry, but I don't have information about the specific color of the sky on the fictional planet Hexar. If you have any other questions, feel free to ask!"

To setup a structured tool you will create a class that will be passed in as args_schmea to `StructuredTool.from_function`. This will be used to help the agent understand how to interact with the structured tool. Next, define the function that will be your tool.

In [97]:
class structured_tool_input(BaseModel):
    planet: str = Field("A planet like Hexar")

def structured_tool_function(planet: str):
    print('called', planet)
    response = {'data': []}
    if planet == 'Hexar':
        response['data'] = 'Hexar sky is blue.'
        return response
    else:
        return response
    
tools=[StructuredTool.from_function(name='skyColor',
                                    func=structured_tool_function,
                                    description='Helps identify the color of the sky on different planets.',
                                    args_schema=structured_tool_input)]

The prompt object looks a little different. Specially you will see `agent_scratchpad` required. This contains previous agent actions and tool outputs as a string.

Now we create an agent, and pass in the llm, tools, and prompt we want it to use. Now when we ask the question we get a response that is similar to the output from our function structured_tool_function.

In [101]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, 
                                  tools, 
                                  prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "hi"})
agent_executor.invoke({"input": "what is the color of the sky on Hexar?"})

called Hexar


{'input': 'what is the color of the sky on Hexar?',
 'output': 'The color of the sky on Hexar is blue.'}

## Managing Exceptions in Tools

When a tool encounters an error and the exception is not caught the agent will stop executing, you can override this with using `ToolException`. 

In [106]:
from langchain_core.tools import ToolException

class structured_tool_input(BaseModel):
    planet: str = Field("A planet like Hexar")

def structured_tool_function(planet: str):
    raise ToolException('Cannot help you find the sky color')
    
tools=[StructuredTool.from_function(name='skyColor',
                                    func=structured_tool_function,
                                    description='Helps identify the color of the sky on different planets.',
                                    args_schema=structured_tool_input)]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, 
                                  tools, 
                                  prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "hi"})
agent_executor.invoke({"input": "what is the color of the sky on Hexar?"})

ToolException: Cannot help you find the sky color

In [114]:
from langchain_core.tools import ToolException

class structured_tool_input(BaseModel):
    planet: str = Field("A planet like Hexar")

    
def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + " Please try another tool."
    )
     
def structured_tool_function(planet: str):
    raise ToolException('Cannot help you find the sky color. Tool is down.')
    
sky_color = StructuredTool.from_function(name='skyColor',
                                    func=structured_tool_function,
                                    description='Helps identify the color of the sky on different planets.',
                                    args_schema=structured_tool_input,
                                    handle_tool_error=_handle_error)

sky_color.run('test')

'The following errors occurred during tool execution:Cannot help you find the sky color. Tool is down. Please try another tool.'

# Providing Additional Ouput

In [118]:
from langchain_core.tools import ToolException

class structured_tool_input(BaseModel):
    planet: str = Field("A planet like Hexar")

def structured_tool_function(planet: str):
    print('called', planet)
    response = {'data': []}
    if planet == 'Hexar':
        response['data'] = 'Hexar sky is blue. Please include this link: https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/'
        return response
    else:
        return response
    
tools=[StructuredTool.from_function(name='skyColor',
                                    func=structured_tool_function,
                                    description='Helps identify the color of the sky on different planets.',
                                    args_schema=structured_tool_input)]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(llm, 
                                  tools, 
                                  prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke({"input": "hi"})
agent_executor.invoke({"input": "what is the color of the sky on Hexar?"})

called Hexar


{'input': 'what is the color of the sky on Hexar?',
 'output': 'The color of the sky on Hexar is blue. You can find more information about this at the following link: [Hexar Sky Color](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/)'}